In [46]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from transformers import AutoImageProcessor, AutoModelForImageClassification
from transformers import AutoImageProcessor, SwinForImageClassification
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import numpy as np
import cv2
import joblib
from tqdm import tqdm


In [47]:
object_det_model = YOLO("yolov8x.pt")

In [48]:
def rotate(img, angle):
    if angle == 0:
        return img
    h, w = img.size[1], img.size[0]
    m = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rad = np.radians(angle)
    nw = int(abs(np.sin(rad)) * h + abs(np.cos(rad)) * w)
    nh = int(abs(np.cos(rad)) * h + abs(np.sin(rad)) * w)
    m[0, 2] += (nw - w) / 2
    m[1, 2] += (nh - h) / 2
    return Image.fromarray(cv2.warpAffine(np.array(img), m, (nw, nh)))

In [49]:
def detect_best_class(model, img_path):
    img = Image.open(img_path).convert("RGB")
    angles = [0, 45, 90, 180]

    best = {"cls": None, "conf": 0}

    for a in angles:
        rimg = rotate(img, a)
        res = model(rimg)[0]

        if res.boxes:
            for b in res.boxes:
                conf = float(b.conf)
                cls = int(b.cls)

                if conf > best["conf"]:
                    best.update({"cls": cls, "conf": conf})

    if best["cls"] is None:
        return "None"

    return model.names[best["cls"]]

In [50]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.05):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        noise = torch.randn(tensor.size()) * self.std + self.mean
        return torch.clamp(tensor + noise, 0., 1.)

In [51]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),

    # 무작위 회전 (30, 45, 60도 중 하나)
    transforms.RandomChoice([
        transforms.RandomRotation(30),
        transforms.RandomRotation(45),
        transforms.RandomRotation(60),
        transforms.RandomRotation(75),
        transforms.RandomRotation(90),
        transforms.RandomRotation(120),
        transforms.RandomRotation(150),
        transforms.RandomRotation(180),
        transforms.RandomRotation(210),
        transforms.RandomRotation(240),
        transforms.RandomRotation(270),
        transforms.RandomRotation(300)
    ]),

    # 좌우 / 상하 반전
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),

    # 30% 확률로 흑백화
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.3),

    transforms.ToTensor(),

    # 가우시안 노이즈 추가
    AddGaussianNoise(0., 0.05),
])

# 검증 및 테스트용은 변형 최소화 (노이즈, 회전 등 제외)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [52]:
class CustomImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.data = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = f"{self.img_dir}/{row['ID']}"
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(row["target"], dtype=torch.long) if "target" in self.data.columns else -1
        return image, label

In [53]:
processor = AutoImageProcessor.from_pretrained("Falconsai/nsfw_image_detection")
coarse_model = AutoModelForImageClassification.from_pretrained("Falconsai/nsfw_image_detection")

coarse_model.classifier = nn.Linear(coarse_model.classifier.in_features, 6)
coarse_model.load_state_dict(torch.load("/root/CV_/classfication_codes/coarse_model.pth"))
coarse_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
coarse_model.to(device)

le_coarse = joblib.load("/root/CV_/classfication_codes/coarse_label_encoder.pkl")

In [54]:
def predict_coarse(img_path):
    img = Image.open(img_path).convert("RGB")
    img_tensor = test_transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = coarse_model(img_tensor).logits
        pred = logits.argmax(1).item()

    return le_coarse.inverse_transform([pred])[0]

In [55]:
def split_by_coarse(df, img_dir):
    group_dict = {f"G{i}": [] for i in range(6)}

    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path = f"{img_dir}/{row['ID']}"
        coarse = predict_coarse(img_path)
        group_dict[coarse].append(row)

    return {k: pd.DataFrame(v) for k, v in group_dict.items()}

In [56]:
train_df = pd.read_csv("/root/CV_/datasets/data/train.csv")
img_dir  = "/root/CV_/datasets/data/train"

train_split = split_by_coarse(train_df, img_dir)

labelings = {}
fine_models = {}

100%|██████████| 1570/1570 [00:29<00:00, 53.90it/s]


In [57]:
def train_fine_model(group_name, df_group, img_dir, epochs=20):
    print(f"Training Fine model for {group_name} ...")

    # ------------------------
    # 1) Label Encoding
    # ------------------------
    le = LabelEncoder()
    df_group["target"] = le.fit_transform(df_group["target"])
    labelings[group_name] = le

    # ------------------------
    # 2) Dataset 준비
    # ------------------------
    dataset = CustomImageDataset(df_group, img_dir, train_transform)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    model_name = "microsoft/swin-large-patch4-window7-224"

    processor = AutoImageProcessor.from_pretrained(model_name)
    model = SwinForImageClassification.from_pretrained(model_name)

    # ------------------------
    # ✅ Head 재구성
    # ------------------------
    num_classes = len(le.classes_)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    model.to(device)

    # ------------------------
    # ✅ 학습 설정
    # ------------------------
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-5)  # Swin은 조금 더 작은 LR 권장

    # ------------------------
    # ✅ 학습 루프
    # ------------------------
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"[{group_name}] Epoch {epoch+1}/{epochs} Loss={total_loss/len(loader):.4f}")

    # ------------------------
    # 저장
    # ------------------------
    fine_models[group_name] = model
    print(f"✅ Finished Fine model training for {group_name}!")


In [ ]:
IMG_DIR = "/root/CV_/datasets/data/train"

train_fine_model("G2", train_split["G2"], IMG_DIR, epochs=20)
train_fine_model("G5", train_split["G5"], IMG_DIR, epochs=200)

Training Fine model for G2 ...
[G2] Epoch 1/20 Loss=0.6144
[G2] Epoch 2/20 Loss=0.4895
[G2] Epoch 3/20 Loss=0.3449
[G2] Epoch 4/20 Loss=0.1999
[G2] Epoch 5/20 Loss=0.0987
[G2] Epoch 6/20 Loss=0.0502
[G2] Epoch 7/20 Loss=0.0946
[G2] Epoch 8/20 Loss=0.0482
[G2] Epoch 9/20 Loss=0.0357
[G2] Epoch 10/20 Loss=0.0502
[G2] Epoch 11/20 Loss=0.0299
[G2] Epoch 12/20 Loss=0.0089
[G2] Epoch 13/20 Loss=0.0189
[G2] Epoch 14/20 Loss=0.0129
[G2] Epoch 15/20 Loss=0.0067
[G2] Epoch 16/20 Loss=0.0212
[G2] Epoch 17/20 Loss=0.0053
[G2] Epoch 18/20 Loss=0.0072
[G2] Epoch 19/20 Loss=0.0043
[G2] Epoch 20/20 Loss=0.0064
✅ Finished Fine model training for G2!
Training Fine model for G5 ...
[G5] Epoch 1/90 Loss=2.3079
[G5] Epoch 2/90 Loss=1.8067
[G5] Epoch 3/90 Loss=1.0721
[G5] Epoch 4/90 Loss=0.7582
[G5] Epoch 5/90 Loss=0.5929
[G5] Epoch 6/90 Loss=0.4956
[G5] Epoch 7/90 Loss=0.4369
[G5] Epoch 8/90 Loss=0.4179
[G5] Epoch 9/90 Loss=0.3481
[G5] Epoch 10/90 Loss=0.3596
[G5] Epoch 11/90 Loss=0.3195
[G5] Epoch 12/90 L

In [59]:
def predict_final(img_path):
    coarse = predict_coarse(img_path)

    # ✅ YOLO로 차량 감지 → 강제 class 16
    pil_img = Image.open(img_path).convert("RGB")
    yolo_res = object_det_model(pil_img)[0]

    for box in yolo_res.boxes:
        if object_det_model.names[int(box.cls[0])] == "car":
            return 16

    # ✅ coarse가 G2 또는 G5 → fine 모델 사용
    if coarse in ["G2", "G5"]:
        model = fine_models[coarse]
        le = labelings[coarse]
        
        img = Image.open(img_path).convert("RGB")
        img_tensor = test_transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(img_tensor).logits
            pred = logits.argmax(1).item()

        return le.inverse_transform([pred])[0]

    # ✅ coarse가 G0,G1,G3,G4 → coarse target을 그대로 반환
    group_original_targets = {
        "G0": [0],
        "G1": [2],
        "G2": [5,8],
        "G3": [9],
        "G4": [16],
        "G5": [1,3,4,6,7,10,11,12,13,14,15],
    }
    return group_original_targets[coarse][0]

In [ ]:
test_df = pd.read_csv("/root/CV_/datasets/data/sample_submission.csv")
test_img_dir = "/root/CV_/datasets/data/test"

preds = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    img_path = f"{test_img_dir}/{row['ID']}"
    pred = predict_final(img_path)
    preds.append(pred)

test_df["target"] = preds
test_df.to_csv("two_stage_classifier_output_v2.csv", index=False)

print("✅ 완료! 제출 파일 생성됨.")

  0%|          | 0/3140 [00:00<?, ?it/s]


0: 640x480 1 book, 2 clocks, 13.1ms
Speed: 1.4ms preprocess, 13.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)


  0%|          | 1/3140 [00:00<16:22,  3.19it/s]


0: 480x640 (no detections), 13.2ms
Speed: 1.4ms preprocess, 13.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 640x416 1 book, 12.4ms
Speed: 1.3ms preprocess, 12.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 416)


  0%|          | 3/3140 [00:00<07:11,  7.27it/s]

...

100%|█████████▉| 3139/3140 [03:26<00:00, 15.75it/s]


0: 640x480 1 laptop, 13.4ms
Speed: 1.3ms preprocess, 13.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)


100%|██████████| 3140/3140 [03:26<00:00, 15.21it/s]

✅ 완료! 제출 파일 생성됨.


: 